# **Fake News Detection Using GPT-3**

This notebook classifies the tweet into fake or real. The dataset is acquired while participating in CONSTRAINT 2021 shared task. For more details about the dataset, please visit the below github link:
https://github.com/parthpatwa/covid19-fake-news-detection

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install openai  #installing openai package

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.8 MB/s eta 0:00:00


In [4]:
import json

OPENAI_API_KEY = ''
with open('/content/drive/My Drive/Colab Notebooks/GPTSumm/OpenAIKey.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    OPENAI_API_KEY = json_data["OPENAI_API_KEY"]

In [5]:
import os
import openai

openai.api_key =  OPENAI_API_KEY

In [6]:
#Since we are making the predictions, therefore, we use validation dataset.
import numpy as np
import pandas as pd

data = pd.read_excel('/content/drive/My Drive/constraint/Constraint_English_Val.xlsx')

In [7]:
data.head(10)

,id,tweet,label
0,1.0,Chinese converting to Islam after realising th...,fake
1,2.0,11 out of 13 people (from the Diamond Princess...,fake
2,3.0,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4.0,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5.0,6/10 Sky's @EdConwaySky explains the latest #C...,real
5,6.0,No one can leave managed isolation for any rea...,real
6,7.0,#IndiaFightsCorona India has one of the lowest...,real
7,8.0,RT @WHO: #COVID19 transmission occurs primaril...,real
8,9.0,News and media outlet ABP Majha on the basis o...,fake
9,10.0,???Church services can???t resume until we???r...,fake


In [8]:
import nltk
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [9]:
data['tweet'] = data['tweet'].apply(lambda x: clean_text(x))

In [10]:
print(len(data))

2140


In [11]:
tweet_list =data['tweet'][:10].tolist()

In [13]:
i =0
tweet_str=''
for tweet in tweet_list:
   tweet_str = tweet_str + str(i) +'.'+'"'+tweet+'"'+'\n'
   i=i+1
   if i ==10:
     break

In [14]:
print(tweet_str)

0."chinese converting to islam after realising that no muslim was affected by coronavirus  in the country"
1." out of  people from the diamond princess cruise ship who had intially tested negative in tests in japan were later confirmed to be positive in the united states"
2." is caused by a bacterium not virus and can be treated with aspirin"
3."mike pence in rnc speech praises donald trump’s  “seamless” partnership with governors and leaves out the presidents state feuds   "
4." skys edconwaysky explains the latest  data and government announcement get more on the coronavirus data here👇  "
5."no one can leave managed isolation for any reason without returning a negative test if they refuse a test they can then be held for a period of up to  days ⁣ ⁣ on june the  exemptions on compassionate grounds have been suspended ⁣ ⁣"
6."indiafightscorona india has one of the lowest  mortality globally with less than  case fatality rate as a result of supervised home isolation amp effective clinic

In [15]:
prompt="Classify the tweets whether they are fake or real:\n" + tweet_str + "\n Tweet detection is done:"
print(prompt)

Classify the tweets whether they are fake or real:
0."chinese converting to islam after realising that no muslim was affected by coronavirus  in the country"
1." out of  people from the diamond princess cruise ship who had intially tested negative in tests in japan were later confirmed to be positive in the united states"
2." is caused by a bacterium not virus and can be treated with aspirin"
3."mike pence in rnc speech praises donald trump’s  “seamless” partnership with governors and leaves out the presidents state feuds   "
4." skys edconwaysky explains the latest  data and government announcement get more on the coronavirus data here👇  "
5."no one can leave managed isolation for any reason without returning a negative test if they refuse a test they can then be held for a period of up to  days ⁣ ⁣ on june the  exemptions on compassionate grounds have been suspended ⁣ ⁣"
6."indiafightscorona india has one of the lowest  mortality globally with less than  case fatality rate as a resul

In [16]:
response = openai.Completion.create(
  model="text-davinci-003",
  prompt= prompt,
  temperature=0,
  max_tokens=120,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

In [18]:
a=(response["choices"][0]["text"]).split('\n')

In [19]:
print(a)

['', '0. Fake', '1. Real', '2. Fake', '3. Real', '4. Real', '5. Real', '6. Real', '7. Fake', '8. Real', '9. Fake']


In [20]:
print(data['label'][0:10])

0    fake
1    fake
2    fake
3    fake
4    real
5    real
6    real
7    real
8    fake
9    fake
Name: label, dtype: object
